# お弁当需要予測大作戦!!(実践編)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font=['MS Gothic'])

from sklearn.linear_model import LinearRegression
from sklearn import metrics

## データの準備

### データの読み込み

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample.csv', header=None)

### データの表示

In [ ]:
train

### DataFrame, Series

In [ ]:
type(train)

In [ ]:
type(train['y'])

## データの分析

### データの基礎統計量を見てみる

In [ ]:
train.describe()

#### yのmin, 20%, 50%, 75%, maxを可視化する

In [ ]:
train[['y']].boxplot()

#### yの中央値と平均値の関係を見てみよう(平均値を赤くしてみる)

In [ ]:
train['y'].plot.hist()
plt.axvline(x=train['y'].mean(), color='red')

#### 曜日ごとの比率を見てみる

In [ ]:
train[['y', 'week']].boxplot(by='week')

#### 中央値と平均値の関係を考察してみる

In [ ]:
train[train['week'] == '金']['y'].plot.hist()
plt.axvline(x=train['y'].mean(), color='red')

In [ ]:
train[train['week'] == '木']['y'].plot.hist()
plt.axvline(x=train['y'].mean(), color='red')

### データを可視化してみる

#### 折れ線グラフ(気温と時間の関係)

In [ ]:
ax = train['temperature'].plot(title='気温')
ax.set_xlabel('time')
ax.set_ylabel('temperature')

#### 折れ線グラフ(yと時間の関係)

In [ ]:
ax = train['y'].plot(title='y')
ax.set_xlabel('time')
ax.set_ylabel('y')

#### 散布図(気温とyの関係)

In [ ]:
train.plot.scatter(x='temperature', y='y', figsize=(5,5))

#### 散布図(kcalとyの関係)

In [ ]:
train.plot.scatter(x='kcal', y='y', figsize=(5,5))

#### ヒストグラム(kcalの頻度)

In [ ]:
train['kcal'].plot.hist()

#### 箱ひげ図

In [ ]:
train[['y', 'weather']].boxplot(by='weather')

### 欠損値を見る

In [ ]:
train.isnull().any()

In [ ]:
train.isnull().sum()

#### こんな例外もある

In [ ]:
train["precipitation"].value_counts()

### 相関関係を見る

In [ ]:
train.corr()

#### 相関がないもの

In [ ]:
train.plot.scatter(x='temperature', y='kcal', figsize=(5,5))

#### 相関があり強いもの

In [ ]:
train.plot.scatter(x='temperature', y='y', figsize=(5,5))

## 単回帰分析(説明変数は1つ)

## 入力データの作成

In [ ]:
X = train['temperature'].values.reshape(-1, 1)
y = train['y']

## 予測モデルの作成

### 予測モデルを作る(LinearRegression)

In [ ]:
model = LinearRegression()

### 予測モデルに学習させる

In [ ]:
model.fit(X, y)

### 傾き

In [ ]:
model.coef_

### 切片

In [ ]:
model.intercept_

### 実際の結果を見てみよう

In [ ]:
train.plot.scatter(x='temperature', y='y')

#回帰直線をプロット
plt.plot(X, model.predict(X), color='red')

### 評価関数RMSEを使って評価してみる

In [ ]:
np.sqrt(metrics.mean_squared_error(y, model.predict(X)))

## 予測してみる

In [ ]:
X_test = test['temperature'].values.reshape(-1, 1)
y_pred = model.predict(X_test)
y_pred

In [ ]:
test['temperature']

### 予測結果をSignateに投稿してみる

In [ ]:
sample[1] = y_pred

In [ ]:
sample.to_csv('submit1.csv', header=None, index=None)

## 重回帰分析(説明変数は2つ以上)

## 入力データの作成

### weekをダミー変数化する

In [ ]:
train['week']

In [ ]:
pd.get_dummies(train['week'])

In [ ]:
X = pd.get_dummies(train[['week', 'temperature']])

## 予測モデルの作成

### 予測モデルを作る

In [ ]:
model = LinearRegression()

### 予測モデルを学習させる

In [ ]:
model.fit(X, y)

### 切片と傾き(重回帰の場合は傾きは複数、切片は1つ)

In [ ]:
model.coef_

In [ ]:
model.intercept_

## 予測してみる

In [ ]:
X_test = pd.get_dummies(test[['week', 'temperature']])

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
sample[1] = y_pred

In [ ]:
sample.to_csv('submit2.csv', header=None, index=None)